In [1]:
# Import libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# Set view options
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

In [3]:
# Read in faults_diagnostics data (1049678 rows)
faults_diagnostics = pd.read_csv('../data/faults_diagnostics.csv', index_col= 'EventTimeStamp', parse_dates = ['EventTimeStamp', 'LocationTimeStamp'])
faults_diagnostics = faults_diagnostics.drop_duplicates()

/var/folders/gj/s4r99xvd4cl789q3qkscy9rr0000gn/T/ipykernel_21939/855893136.py:2: DtypeWarning: Columns (17,18,21,22,24,25,26,27,28,29,30,31,33,37,38,39,40) have mixed types. Specify dtype option on import or set low_memory=False.
  faults_diagnostics = pd.read_csv('../data/faults_diagnostics.csv', index_col= 'EventTimeStamp', parse_dates = ['EventTimeStamp', 'LocationTimeStamp'])


In [4]:
faults_diagnostics['PartialDerate'] = (faults_diagnostics['spn'] == 1569)
faults_diagnostics['FullDerate'] = (faults_diagnostics['spn'] == 5246)

rolling_7D = faults_diagnostics.loc[::-1].groupby('EquipmentID', as_index = False)[['PartialDerate', 'FullDerate']].rolling('7D').sum()[::-1]
rolling_7D.loc[rolling_7D['PartialDerate'] > 0, 'PartialDerate'] = 1
rolling_7D.loc[rolling_7D['FullDerate'] > 0, 'FullDerate'] = 1

rolling_7D['PartialDerate'] = ((rolling_7D['EquipmentID'].shift() != rolling_7D['EquipmentID']) | (rolling_7D['PartialDerate'].shift() != rolling_7D['PartialDerate'])).cumsum()
rolling_7D['FullDerate'] = ((rolling_7D['EquipmentID'].shift() != rolling_7D['EquipmentID']) | (rolling_7D['FullDerate'].shift() != rolling_7D['FullDerate'])).cumsum()

faults_diagnostics = faults_diagnostics.drop(columns = ['PartialDerate', 'FullDerate'])
faults_diagnostics = faults_diagnostics.merge(rolling_7D, on = ['EventTimeStamp', 'EquipmentID'])
faults_diagnostics = faults_diagnostics.sort_values(['EquipmentID', 'EventTimeStamp']).drop_duplicates()

In [5]:
partial_derate_clusters = faults_diagnostics.loc[faults_diagnostics['PartialDerate'].isin(faults_diagnostics.loc[faults_diagnostics['spn'] == 1569, 'PartialDerate'])]
partial_derate_mask = partial_derate_clusters.groupby('EquipmentID')['spn'].apply(lambda x: x.shift().eq(1569).cumsum().eq(0)).to_list()
partial_derates = partial_derate_clusters[partial_derate_mask].reset_index()

partial_derates['DeltaT'] = partial_derates.loc[::-1].groupby(['EquipmentID', 'PartialDerate'])['EventTimeStamp'].diff().dt.total_seconds().div(3600)[::-1].abs()
partial_derates['DeltaT'] = partial_derates.loc[::-1].groupby(['EquipmentID', 'PartialDerate'])['DeltaT'].cumsum()[::-1]
partial_derates = partial_derates.loc[partial_derates['DeltaT'] != 0].set_index('EventTimeStamp').drop_duplicates(subset = ['EquipmentID', 'PartialDerate', 'spn'], keep = 'last')
partial_derates['DeltaT'] = partial_derates['DeltaT'].fillna(0)

partial_derates.groupby('spn')['DeltaT'].describe().sort_values('count', ascending = False)

,count,mean,std,min,25%,50%,75%,max
spn,,,,,,,,
1569,491.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3362,95.0,1.770570,5.530648,0.055556,0.999722,1.000000,1.000833,52.859722
111,94.0,35.240287,50.043673,0.026389,0.961458,6.269306,52.806458,164.119722
1761,70.0,7.867294,21.247722,0.000278,0.000278,0.124306,2.648750,110.734444
3251,54.0,15.923508,23.228611,0.000278,2.328403,4.492639,20.735764,123.701111
3216,48.0,20.974115,34.054952,0.002778,1.177500,5.112361,23.184583,149.280000
3226,45.0,29.574272,43.513908,0.210278,1.023889,9.063333,37.613889,164.225556
596,38.0,55.224444,44.688077,0.758611,17.968542,48.519861,96.655556,146.443056
641,35.0,15.581127,26.790030,0.195556,1.195417,2.885556,15.991111,101.368056


In [6]:
full_derate_clusters = faults_diagnostics.loc[faults_diagnostics['FullDerate'].isin(faults_diagnostics.loc[faults_diagnostics['spn'] == 5246, 'FullDerate'])]
full_derate_mask = full_derate_clusters.groupby('EquipmentID')['spn'].apply(lambda x: x.shift().eq(5246).cumsum().eq(0)).to_list()
full_derates = full_derate_clusters[full_derate_mask].reset_index()

full_derates['DeltaT'] = full_derates.loc[::-1].groupby(['EquipmentID', 'FullDerate'])['EventTimeStamp'].diff().dt.total_seconds().div(3600)[::-1].abs()
full_derates['DeltaT'] = full_derates.loc[::-1].groupby(['EquipmentID', 'FullDerate'])['DeltaT'].cumsum()[::-1]
full_derates = full_derates.loc[full_derates['DeltaT'] != 0].set_index('EventTimeStamp').drop_duplicates(subset = ['EquipmentID', 'FullDerate', 'spn'], keep = 'last')
full_derates['DeltaT'] = full_derates['DeltaT'].fillna(0)

full_derates.groupby('spn')['DeltaT'].describe().sort_values('count', ascending = False)

,count,mean,std,min,25%,50%,75%,max
spn,,,,,,,,
5246,191.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1569,151.0,12.849152,14.843403,1.018333,4.000556,6.621667,16.364306,99.539722
111,44.0,25.889766,41.801047,0.028056,1.786181,7.511528,26.580972,163.804444
3362,40.0,15.913681,19.943055,0.114722,3.848681,7.912917,19.109792,100.540000
1761,38.0,18.371323,28.780975,0.000278,4.469722,5.641389,20.062569,147.710278
3216,37.0,24.989797,20.078970,0.174722,11.695000,21.151111,32.138333,70.280000
3226,35.0,30.162119,34.548427,0.000278,5.863194,16.865833,32.587778,132.666111
3031,34.0,13.469739,15.828093,1.175556,4.998750,7.134306,16.715903,69.679167
3364,29.0,8.636073,7.692951,0.309722,4.587778,5.485278,11.817222,34.015278


In [ ]:
event_timestamps = faults_diagnostics.reset_index()[['EventTimeStamp','EquipmentID']].drop_duplicates()
event_timestamps['DeltaT'] = event_timestamps.groupby('EquipmentID')['EventTimeStamp'].diff(-1).dt.total_seconds().div(3600).abs()
event_timestamps = faults_diagnostics.merge(event_timestamps, on = ['EventTimeStamp', 'EquipmentID'])
event_timestamps = event_timestamps[event_timestamps['DeltaT'] <= 168]

In [ ]:
fig, ax = plt.subplots(figsize = (8, 5), dpi = 300)
sns.histplot(event_timestamps['DeltaT'])
plt.xticks([0,24,48,72,96,120,144,168])
ax.set_yscale('log', base = 2)
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize = (8, 5), dpi = 300)
sns.histplot(event_timestamps[event_timestamps['spn'] == 1569]['DeltaT'])
plt.xticks([0,24,48,72,96,120,144,168])
ax.set_yscale('log', base = 2)
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize = (8, 5), dpi = 300)
sns.histplot(event_timestamps[event_timestamps['spn'] == 5246]['DeltaT'])
plt.xticks([0,24,48,72,96,120,144,168])
ax.set_yscale('log', base = 2)
plt.show()

In [ ]:
spn_crosstab = faults_diagnostics
spn_crosstab['prev_spn'] = spn_crosstab.groupby(['EquipmentID'])['spn'].shift(-1)
spn_crosstab = spn_crosstab.dropna(subset = 'prev_spn')
spn_crosstab = pd.crosstab(spn_crosstab['spn'], spn_crosstab['prev_spn'].astype(int))
spn_crosstab.style.background_gradient(cmap = 'Blues')